In [1]:
from AlgorithmImports import QCAlgorithm
from IndicatorsImports import *
from DataImports import *
from OrdersImports import *
from datetime import datetime, timedelta

class EminiSP500FuturesAlgo(QCAlgorithm):
    def Initialize(self):
        self.symbol = "ES"
        self.resolution = Resolution.Minute
        self.rsiPeriod = 14
        self.stochasticPeriod = 14
        self.meanReversionLookback = 20
        self.macdFastPeriod = 12
        self.macdSlowPeriod = 26
        self.macdSignalPeriod = 9
        self.holdingPeriod = timedelta(minutes=30)
        self.lastTradeTime = datetime.min

        self.stochastic = self.STO(self.symbol, self.stochasticPeriod, self.resolution, 3, 3, MovingAverageType.Wilders, MovingAverageType.Wilders)
        self.rsi = self.RSI(self.symbol, self.rsiPeriod, self.resolution)
        self.meanReversion = self.SMA(self.symbol, self.meanReversionLookback, self.resolution)
        self.macd = self.MACD(self.symbol, self.macdFastPeriod, self.macdSlowPeriod, self.macdSignalPeriod, MovingAverageType.Wilders, self.resolution)

    def OnData(self, data):
        if not self.Portfolio[self.symbol].Invested and self.Time - self.lastTradeTime < self.holdingPeriod:
            return

        if self.stochastic.IsReady and self.rsi.IsReady and self.macd.IsReady and self.meanReversion.IsReady:
            currentPrice = self.Securities[self.symbol].Price
            mean = self.meanReversion.Current.Value

            if self.ShouldEnterLongPosition(currentPrice, mean) and self.macd.Current.Value > self.macd.Signal.Current.Value:
                self.SetHoldings(self.symbol, 1.0)
                self.lastTradeTime = self.Time
            elif self.ShouldEnterShortPosition(currentPrice, mean) and self.macd.Current.Value < self.macd.Signal.Current.Value:
                self.SetHoldings(self.symbol, -1.0)
                self.lastTradeTime = self.Time
            elif self.Portfolio[self.symbol].Invested and self.Time - self.lastTradeTime >= self.holdingPeriod:
                self.Liquidate(self.symbol)
                self.lastTradeTime = self.Time

    def ShouldEnterLongPosition(self, currentPrice, mean):
        return self.stochastic.StochK.Current.Value < 20 and self.rsi.Current.Value < 30 and currentPrice < mean

    def ShouldEnterShortPosition(self, currentPrice, mean):
        return self.stochastic.StochK.Current.Value > 80 and self.rsi.Current.Value > 70 and currentPrice > mean


ImportError: cannot import name 'QCAlgorithm' from 'AlgorithmImports' (unknown location)